In [1]:
import numpy as np
from dlordinal.datasets import FGNet
from dlordinal.losses import TriangularLoss
from dlordinal.metrics import amae, mmae
from skorch import NeuralNetClassifier
from torch import nn
from torch.optim import Adam
from torchvision import models
from torchvision.transforms import Compose, ToTensor

# Download the FGNet dataset
fgnet_train = FGNet(
    root="./datasets",
    train=True,
    #强制转换为 int64）
    target_transform=lambda x: np.array(x, dtype=np.int64),
    transform=Compose([ToTensor()]),
)
fgnet_test = FGNet(
    root="./datasets",
    train=False,
    #强制转换为 int64）
    target_transform=lambda x: np.array(x, dtype=np.int64),
    transform=Compose([ToTensor()]),
)

num_classes_fgnet = len(fgnet_train.classes)

# Model
model = models.resnet18(weights="IMAGENET1K_V1")
model.fc = nn.Linear(model.fc.in_features, num_classes_fgnet)

# Loss function
loss_fn = TriangularLoss(base_loss=nn.CrossEntropyLoss(), num_classes=num_classes_fgnet)

# Skorch estimator
estimator = NeuralNetClassifier(
    module=model,
    criterion=loss_fn,
    optimizer=Adam,
    lr=1e-3,
    max_epochs=25,
)

estimator.fit(X=fgnet_train, y=fgnet_train.targets)
train_probs = estimator.predict_proba(fgnet_train)
test_probs = estimator.predict_proba(fgnet_test)

# Metrics
amae_metric = amae(np.array(fgnet_test.targets), test_probs)
mmae_metric = mmae(np.array(fgnet_test.targets), test_probs)
print(f"Test AMAE: {amae_metric}, Test MMAE: {mmae_metric}")

Files already downloaded and verified
Files already processed and verified
Files already split and verified
Files already downloaded and verified
Files already processed and verified
Files already split and verified
  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1        1.7190       0.3168        1.7735  27.6544
      2        0.9176       0.3602        1.7816  22.2978
      3        0.5885       0.5093        1.4999  22.1870
      4        0.4544       0.4658        1.9440  23.4645
      5        0.3989       0.5776        1.1674  22.7371
      6        0.3826       0.5776        1.1716  22.6531
      7        0.3633       0.5963        1.1281  22.1734
      8        0.3565       0.6211        1.0612  22.7198
      9        0.3518       0.6149        1.0575  22.2039
     10        0.3492       0.6025        1.0387  21.9576
     11        0.3476       0.6211        1.0418  22.3816
     12        0.3464       0.